In [13]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import json
from SPARQLWrapper import SPARQLWrapper, JSON
from surprise import SVD
from surprise import Reader
from surprise import Dataset
from surprise import evaluate, print_perf
from surprise import accuracy
from surprise.prediction_algorithms.knns import *
%matplotlib inline

In [272]:
ratings = pd.read_csv( '/Users/jose/Documents/Maestria/Sistemas de recomendación/Taller 3/ml-latest/ratings.csv', sep = ',', engine = 'python' )

In [78]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [4]:
ratings.count()

userId       26024289
movieId      26024289
rating       26024289
timestamp    26024289
dtype: int64

In [43]:
movies = pd.read_csv( '/Users/jose/Documents/Maestria/Sistemas de recomendación/Taller 3/ml-latest/movies.csv', sep = ',', engine = 'python' )

In [6]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
movies.count()

movieId    45843
title      45843
genres     45843
dtype: int64

In [40]:
tags = pd.read_csv( '/Users/jose/Documents/Maestria/Sistemas de recomendación/Taller 3/ml-latest/tags.csv', sep = ',', engine = 'python' )

In [9]:
tags.head()

,userId,movieId,tag,timestamp
0,1,318,narrated,1425942391
1,20,4306,Dreamworks,1459855607
2,20,89302,England,1400778834
3,20,89302,espionage,1400778836
4,20,89302,jazz,1400778841


In [10]:
tags.count()

userId       753170
movieId      753170
tag          753154
timestamp    753170
dtype: int64

In [65]:
links = pd.read_csv( '/Users/jose/Documents/Maestria/Sistemas de recomendación/Taller 3/ml-latest/links.csv', sep = ',', engine = 'python' )

In [66]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [13]:
links.count()

movieId    45843
imdbId     45843
tmdbId     45624
dtype: int64

In [14]:
genomeScores = pd.read_csv( '/Users/jose/Documents/Maestria/Sistemas de recomendación/Taller 3/ml-latest/genome-scores.csv', sep = ',', engine = 'python' )

In [15]:
genomeScores.head()

,movieId,tagId,relevance
0,1,1,0.02475
1,1,2,0.02475
2,1,3,0.04900
3,1,4,0.07750
4,1,5,0.12450


In [16]:
genomeScores.count()

movieId      12400104
tagId        12400104
relevance    12400104
dtype: int64

In [17]:
genomeTags = pd.read_csv( '/Users/jose/Documents/Maestria/Sistemas de recomendación/Taller 3/ml-latest/genome-tags.csv', sep = ',', engine = 'python' )

In [18]:
genomeTags.head()

,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s


In [19]:
genomeTags.count()

tagId    1128
tag      1128
dtype: int64

In [41]:
tags['DateTime'] = pd.to_datetime(tags['timestamp'], unit='s')

In [21]:
tags.head()

,userId,movieId,tag,timestamp,DateTime
0,1,318,narrated,1425942391,2015-03-09 23:06:31
1,20,4306,Dreamworks,1459855607,2016-04-05 11:26:47
2,20,89302,England,1400778834,2014-05-22 17:13:54
3,20,89302,espionage,1400778836,2014-05-22 17:13:56
4,20,89302,jazz,1400778841,2014-05-22 17:14:01


In [22]:
ratings['DateTime'] = pd.to_datetime(ratings['timestamp'], unit='s')

In [23]:
ratings.head()

,userId,movieId,rating,timestamp,DateTime
0,1,110,1.0,1425941529,2015-03-09 22:52:09
1,1,147,4.5,1425942435,2015-03-09 23:07:15
2,1,858,5.0,1425941523,2015-03-09 22:52:03
3,1,1221,5.0,1425941546,2015-03-09 22:52:26
4,1,1246,5.0,1425941556,2015-03-09 22:52:36


In [24]:
date_rating = pd.DataFrame(ratings['DateTime']).drop_duplicates()

In [25]:
date_rating.describe()

,DateTime
count,20549435
unique,20549435
top,2013-04-24 11:28:21
freq,1
first,1995-01-09 11:46:44
last,2017-08-04 06:57:50


In [26]:
users_rating = pd.DataFrame(ratings['userId']).drop_duplicates()

In [27]:
users_rating.describe()

,userId
count,270896.000000
mean,135448.500000
std,78201.083599
min,1.000000
25%,67724.750000
50%,135448.500000
75%,203172.250000
max,270896.000000


In [28]:
genomes = pd.merge(genomeScores,genomeTags, on=['tagId'])

In [29]:
genomes.head()

,movieId,tagId,relevance,tag
0,1,1,0.02475,007
1,2,1,0.03750,007
2,3,1,0.03800,007
3,4,1,0.03125,007
4,5,1,0.04100,007


In [30]:
genomes.count()

movieId      12400104
tagId        12400104
relevance    12400104
tag          12400104
dtype: int64

In [31]:
pelicula = pd.DataFrame(tags[tags['movieId']==1]['tag']).drop_duplicates()
pelicula.sort_values('tag')

,tag
657468,2009 reissue in Stereoscopic 3-D
614820,3D
657469,55 movies every kid should see--Entertainment ...
736440,American Animation
355068,Animation
657470,BD-Video
706177,Best of Rotten Tomatoes: All Time
686606,Buzz Lightyear
686607,CG animation
510255,CGI


In [44]:
pelicula = movies[movies['movieId']==18]
pelicula

,movieId,title,genres
17,18,Four Rooms (1995),Comedy


In [48]:
def get_movie_description():
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql.setReturnFormat(JSON)
    total_results = 16
    offset = 0
    out = []
 
    while total_results > 0:
            sparql.setQuery("""
            SELECT ?label ?film ?Rdate ?imdb ?country ?director 
            WHERE { ?film rdf:type dbo:Film . 
                    OPTIONAL { ?film rdfs:label ?label }. 
                    OPTIONAL { ?film dbo:releaseDate ?Rdate }.
                    OPTIONAL { ?film dbo:imdbId ?imdb }. 
                    OPTIONAL { ?film dbp:country ?country }. 
                    OPTIONAL { ?film dbo:director ?director }. 
            FILTER (lang(?label) = \'en\') } 
            OFFSET """ + str(offset) + """
            LIMIT 10000""")

            result = sparql.query()
            offset = offset + 10000
            
            processed_results = json.load(result.response)
            cols = processed_results['head']['vars']

            for row in processed_results['results']['bindings']:
                item = []
                for c in cols:
                    item.append(row.get(c, {}).get('value'))
                out.append(item)

            total_results=total_results-1
    resultM = pd.DataFrame(out, columns=cols)
    return resultM

In [67]:
resultDBPEDIA = get_movie_description()
resultDBPEDIA

,label,film,Rdate,imdb,country,director
0,THX 1138,http://dbpedia.org/resource/THX_1138,1971-03-11,0066434,United States,http://dbpedia.org/resource/George_Lucas
1,The Big Fisherman,http://dbpedia.org/resource/The_Big_Fisherman,1959-08-04,0052627,United States,http://dbpedia.org/resource/Frank_Borzage
2,The Hollywood Revue of 1929,http://dbpedia.org/resource/The_Hollywood_Revu...,1929-06-20,0019993,United States,http://dbpedia.org/resource/Charles_Reisner
3,The War Game,http://dbpedia.org/resource/The_War_Game,1965-11-01,0059894,UK,http://dbpedia.org/resource/Peter_Watkins
4,Bad Company (1972 film),http://dbpedia.org/resource/Bad_Company_(1972_...,1972-10-08,0068245,United States,http://dbpedia.org/resource/Robert_Benton
5,Bad Company (1999 film),http://dbpedia.org/resource/Bad_Company_(1999_...,1999-10-20,0211504,France,http://dbpedia.org/resource/Jean-Pierre_Améris
6,Blown Away (1994 film),http://dbpedia.org/resource/Blown_Away_(1994_f...,1994-07-01,0109303,United States,http://dbpedia.org/resource/Stephen_Hopkins_(d...
7,Coup de Grâce (1969 film),http://dbpedia.org/resource/Coup_de_Grâce_(196...,1969-10-02,0183898,Argentina,http://dbpedia.org/resource/Ricardo_Becher
8,God Is My Co-Pilot (film),http://dbpedia.org/resource/God_Is_My_Co-Pilot...,1945-02-21,0037746,United States,http://dbpedia.org/resource/Robert_Florey
9,Jack of All Trades (1936 film),http://dbpedia.org/resource/Jack_of_All_Trades...,1936-12-30,0030905,United Kingdom,http://dbpedia.org/resource/Robert_Stevenson_(...


In [50]:
resultDBPEDIA.describe()

,label,film,Rdate,imdb,country,director
count,144319,144319,26180,34952,97615,108208
unique,129591,129591,15285,30676,1210,27997
top,Bridges of Sarajevo,http://dbpedia.org/resource/Bridges_of_Sarajevo,2001-12-12,0055747,United States,http://dbpedia.org/resource/D._W._Griffith
freq,78,78,18,25,40101,279


In [47]:
film_date = pd.DataFrame(resultDBPEDIA['Rdate'])
film_date2=film_date.drop_duplicates()
fd=film_date.loc[film_date['Rdate'].isnull()]
fd

,Rdate
59,None
60,None
61,None
62,None
63,None
64,None
65,None
66,None
67,None
68,None


In [68]:
resultDBPEDIA['imdbId'] = pd.to_numeric(resultDBPEDIA.imdb, errors='coerce')
del resultDBPEDIA['imdb']

In [69]:
resultDBPEDIA

,label,film,Rdate,country,director,imdbId
0,THX 1138,http://dbpedia.org/resource/THX_1138,1971-03-11,United States,http://dbpedia.org/resource/George_Lucas,66434.0
1,The Big Fisherman,http://dbpedia.org/resource/The_Big_Fisherman,1959-08-04,United States,http://dbpedia.org/resource/Frank_Borzage,52627.0
2,The Hollywood Revue of 1929,http://dbpedia.org/resource/The_Hollywood_Revu...,1929-06-20,United States,http://dbpedia.org/resource/Charles_Reisner,19993.0
3,The War Game,http://dbpedia.org/resource/The_War_Game,1965-11-01,UK,http://dbpedia.org/resource/Peter_Watkins,59894.0
4,Bad Company (1972 film),http://dbpedia.org/resource/Bad_Company_(1972_...,1972-10-08,United States,http://dbpedia.org/resource/Robert_Benton,68245.0
5,Bad Company (1999 film),http://dbpedia.org/resource/Bad_Company_(1999_...,1999-10-20,France,http://dbpedia.org/resource/Jean-Pierre_Améris,211504.0
6,Blown Away (1994 film),http://dbpedia.org/resource/Blown_Away_(1994_f...,1994-07-01,United States,http://dbpedia.org/resource/Stephen_Hopkins_(d...,109303.0
7,Coup de Grâce (1969 film),http://dbpedia.org/resource/Coup_de_Grâce_(196...,1969-10-02,Argentina,http://dbpedia.org/resource/Ricardo_Becher,183898.0
8,God Is My Co-Pilot (film),http://dbpedia.org/resource/God_Is_My_Co-Pilot...,1945-02-21,United States,http://dbpedia.org/resource/Robert_Florey,37746.0
9,Jack of All Trades (1936 film),http://dbpedia.org/resource/Jack_of_All_Trades...,1936-12-30,United Kingdom,http://dbpedia.org/resource/Robert_Stevenson_(...,30905.0


In [70]:
dataOntology = pd.merge(links,resultDBPEDIA, on=['imdbId'])

In [71]:
dataOntology

,movieId,imdbId,tmdbId,label,film,Rdate,country,director
0,9,114576,9091.0,Sudden Death (1995 film),http://dbpedia.org/resource/Sudden_Death_(1995...,None,United States,http://dbpedia.org/resource/Peter_Hyams
1,11,112346,9087.0,The American President,http://dbpedia.org/resource/The_American_Presi...,None,United States,http://dbpedia.org/resource/Rob_Reiner
2,18,113101,5.0,Four Rooms,http://dbpedia.org/resource/Four_Rooms,None,United States,http://dbpedia.org/resource/Allison_Anders
3,18,113101,5.0,Four Rooms,http://dbpedia.org/resource/Four_Rooms,None,United States,http://dbpedia.org/resource/Alexandre_Rockwell
4,18,113101,5.0,Four Rooms,http://dbpedia.org/resource/Four_Rooms,None,United States,http://dbpedia.org/resource/Quentin_Tarantino
5,18,113101,5.0,Four Rooms,http://dbpedia.org/resource/Four_Rooms,None,United States,http://dbpedia.org/resource/Robert_Rodriguez
6,22,112722,1710.0,Copycat (film),http://dbpedia.org/resource/Copycat_(film),1995-10-27,United States,http://dbpedia.org/resource/Jon_Amiel
7,26,114057,16420.0,Othello (1995 film),http://dbpedia.org/resource/Othello_(1995_film),1995-12-15,None,http://dbpedia.org/resource/Oliver_Parker
8,27,114011,9263.0,Now and Then (film),http://dbpedia.org/resource/Now_and_Then_(film),None,United States,http://dbpedia.org/resource/Lesli_Linka_Glatter
9,28,114117,17015.0,Persuasion (1995 film),http://dbpedia.org/resource/Persuasion_(1995_f...,None,United Kingdom,http://dbpedia.org/resource/Roger_Michell


http://idevji.com/building-your-first-recommender-with-python-and-spark/

pip install pyspark

In [324]:
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
from pyspark import SparkContext
from pyspark.sql import Row

In [327]:
sc = SparkContext()

In [341]:
movielens = sc.textFile("/Users/jose/Documents/Maestria/Sistemas de recomendación/Taller 3/ml-latest/ratings2.csv")

In [312]:
movielens.count()

26024289

In [342]:
#Clean up the data by splitting it, 
#movielens readme says the data is split by tabs and
#is user product rating timestamp
clean_data = movielens.map(lambda x:x.split(','))

In [343]:
#We'll need to map the movielens data to a Ratings object 
#A Ratings object is made up of (user, item, rating)
mls = movielens.map(lambda l: l.split(','))
ratingsRDD = mls.map(lambda x: Rating( int(x[0]), int(x[1]), float(x[2])))

In [344]:
#Setting up the parameters for ALS
rank = 5 # Latent Factors to be made
numIterations = 10 # Times to repeat process

In [345]:
#Need a training and test set, test set is not used in this example.
train, test = ratingsRDD.randomSplit([0.7,0.3],7856)

In [346]:
train.count()

18219046

In [260]:
test.count()

7805243

In [348]:
#Create the model on the training data
model = ALS.train(train, rank, numIterations)

In [262]:
# For Product X, Find N Users to Sell To
model.recommendUsers(242,10)

[Rating(user=62393, product=242, rating=9.544833083584434),
 Rating(user=265420, product=242, rating=8.191496698015467),
 Rating(user=10734, product=242, rating=7.953971715794993),
 Rating(user=74678, product=242, rating=7.803197609685621),
 Rating(user=160201, product=242, rating=7.777555747438187),
 Rating(user=80324, product=242, rating=7.615517031361526),
 Rating(user=48830, product=242, rating=7.598116647464171),
 Rating(user=180114, product=242, rating=7.416079164304811),
 Rating(user=51406, product=242, rating=7.390228686035627),
 Rating(user=188762, product=242, rating=7.388653391494319)]

In [263]:
# For User Y Find N Products to Promote
model.recommendProducts(196,10)

[Rating(user=196, product=116821, rating=16.74403511150999),
 Rating(user=196, product=6085, rating=15.0558367105263),
 Rating(user=196, product=26968, rating=14.70983819069459),
 Rating(user=196, product=127126, rating=14.184580468342025),
 Rating(user=196, product=87294, rating=13.680092661660542),
 Rating(user=196, product=106436, rating=13.622039283199666),
 Rating(user=196, product=145468, rating=11.811057178921637),
 Rating(user=196, product=132296, rating=11.37427410060872),
 Rating(user=196, product=135755, rating=11.264696035699487),
 Rating(user=196, product=133203, rating=11.196962118220156)]

In [264]:
#Predict Single Product for Single User
model.predict(196, 242)

3.5820829483440093

http://www.learnbymarketing.com/644/recsys-pyspark-als/

In [265]:
# Predict Multi Users and Multi Products
# Pre-Processing
pred_input = train.map(lambda x:(x[0],x[1]))   

In [266]:
# Lots of Predictions
#Returns Ratings(user, item, prediction)
pred = model.predictAll(pred_input) 

In [267]:
#Get Performance Estimate
#Organize the data to make (user, product) the key)
true_reorg = train.map(lambda x:((x[0],x[1]), x[2]))
pred_reorg = pred.map(lambda x:((x[0],x[1]), x[2]))

In [268]:
#Do the actual join
true_pred = true_reorg.join(pred_reorg)

In [269]:
#Need to be able to square root the Mean-Squared Error
from math import sqrt

MSE = true_pred.map(lambda r: (r[1][0] - r[1][1])**2).mean()
RMSE = sqrt(MSE)
RMSE

0.7615977884685018

In [270]:
#Test Set Evaluation
test_input = test.map(lambda x:(x[0],x[1])) 
pred_test = model.predictAll(test_input)
test_reorg = test.map(lambda x:((x[0],x[1]), x[2]))
pred_reorg = pred_test.map(lambda x:((x[0],x[1]), x[2]))
test_pred = test_reorg.join(pred_reorg)
test_MSE = test_pred.map(lambda r: (r[1][0] - r[1][1])**2).mean()
test_RMSE = sqrt(test_MSE)
test_RMSE

0.8396319358378136

https://community.hortonworks.com/articles/81359/pyspark-streaming-wordcount-example.html

In [222]:
import sys
import os
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
batchIntervalSeconds = 3 
 

def creatingFunc():
  ssc = StreamingContext(sc, batchIntervalSeconds)
  # Set each DStreams in this context to remember RDDs it generated in the last given duration.
  # DStreams remember RDDs only for a limited duration of time and releases them for garbage
  # collection. This method allows the developer to specify how long to remember the RDDs (
  # if the developer wishes to query old data outside the DStream computation).
  ssc.remember(6)
  
  lines = ssc.textFileStream('StreamInput/')
  lines.pprint()
 
  words = lines.flatMap(lambda line: line.split(","))
  pairs = words.map(lambda word: (word, 1))
  wordCounts = pairs.reduceByKey(lambda x, y: x + y)
 
  def process(time, rdd):
    df = sqlContext.createDataFrame(rdd)
    df.registerTempTable("myCounts")
  
  wordCounts.foreachRDD(process)
  
  return ssc

In [223]:
checkpointDir = "StreamOutput/"
ssc = StreamingContext.getActiveOrCreate(checkpointDir, creatingFunc)

In [224]:
# This line starts the streaming context in the background.
ssc.start()

-------------------------------------------
Time: 2018-05-20 15:11:54
-------------------------------------------

-------------------------------------------
Time: 2018-05-20 15:11:57
-------------------------------------------

-------------------------------------------
Time: 2018-05-20 15:12:00
-------------------------------------------

-------------------------------------------
Time: 2018-05-20 15:12:03
-------------------------------------------

-------------------------------------------
Time: 2018-05-20 15:12:06
-------------------------------------------

-------------------------------------------
Time: 2018-05-20 15:12:09
-------------------------------------------

-------------------------------------------
Time: 2018-05-20 15:12:12
-------------------------------------------

-------------------------------------------
Time: 2018-05-20 15:12:15
-------------------------------------------

-------------------------------------------
Time: 2018-05-20 15:12:18
----------

In [294]:
if( not sc._jvm.StreamingContext.getActive().isEmpty() ): 
	sc._jvm.StreamingContext.getActive().get().stop(False)

In [326]:
sc.stop()

In [383]:
from flurs.datasets import fetch_movielens
from flurs.recommender.fm import FMRecommender
from flurs.evaluator import Evaluator

import logging
import os
import sys

In [411]:
data = fetch_movielens(data_home="/Users/jose/Documents/Maestria/Sistemas de recomendación/Laboratorio 1/ml-100k", size='100k')

In [412]:
data

{'can_repeat': False,
 'contexts': {'item': 18, 'others': 32, 'user': 23},
 'n_item': 1172,
 'n_sample': 21201,
 'n_user': 928,
 'samples': [<flurs.data.entity.Event at 0x1a2c76db70>,
  ...]}

In [413]:
 rec = FMRecommender(p=sum(data.contexts.values()),  # number of dimensions of input vector
                        k=40,
                        l2_reg_w0=2.,
                        l2_reg_w=8.,
                        l2_reg_V=16.,
                        learn_rate=.004)

In [475]:
rec = FMRecommender(p=8,k=2)

In [476]:
rec.initialize()

In [477]:
evaluator = Evaluator(rec, data.can_repeat)

In [478]:
n_batch_train = int(data.n_sample * 0.2)  # 20% for pre-training to avoid cold-start
n_batch_test = int(data.n_sample * 0.1)  # 10% for evaluation of pre-training
batch_tail = n_batch_train + n_batch_test

In [479]:
# pre-train
# 20% for batch training | 10% for batch evaluate
# after the batch training, 10% samples are used for incremental updating
logging.info('batch pre-training before streaming input')

In [480]:
evaluator.fit(
        data.samples[:n_batch_train],
        data.samples[n_batch_train:batch_tail],
        n_epoch=1  # single pass even for batch training
    )

ValueError: shapes (2,8) and (73,1) not aligned: 8 (dim 1) != 73 (dim 0)

In [481]:
# 70% incremental evaluation and updating
logging.info('incrementally predict, evaluate and update the recommender')
res = evaluator.evaluate(data.samples[batch_tail:])

In [482]:
print(res)

<generator object Evaluator.evaluate at 0x1a1a028fc0>


In [502]:
from flurs.data.entity import User, Item

# define a user with index 0
user = User(0)

# define an item with index 0
item = Item(0)

# interaction between a user and item
event = Event(user, item)

In [484]:
user = User(0, feature=np.array([0,0,1]))
item = Item(0, feature=np.array([2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]))
event = Event(user, item, context=np.array([0,4,0]))

In [485]:
rec.register(user)

In [486]:
rec.register(item)

In [487]:
rec.recommend(user, np.array([0]),context=np.array([0,4,0]))

ValueError: dimension mismatch

In [488]:
user_a = User( 0, feature = np.array([0, 0, 1]))

In [489]:
user_b = User( 1, feature = np.array([1, 0, 1]))

In [490]:
user_c = User( 2, feature = np.array([1, 3, 1]))

In [497]:
item_a = Item( 0, feature = np.array([2, 1, 1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]))

In [498]:
item_b = Item( 1, feature = np.array([0, 2, 1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]))

In [503]:
rec.register(user_a)
rec.register(user_b)
rec.register(user_c)

In [504]:
rec.register(item_a)
rec.register(item_b)

In [505]:
rec.update(Event(user_a, item_b, context=np.array([1,1])))

ValueError: shapes (2,8) and (23,1) not aligned: 8 (dim 1) != 23 (dim 0)